In [1]:
def s(x): return " "*(10+x)
print(s(5)+".\n"+s(4)+"..:\n"+s(2)+"Hultnér\n"+s(0)+"Technologies\n\n@ahultner | https://hultner.se/")

               .
              ..:
            Hultnér
          Technologies

@ahultner | https://hultner.se/


# ⠠⠵ Schema-based-API-Testing
**Automatically generate test-cases based on your API-schemas.**  
Shorter intro text.


## Index
- Short introduction to API-schemas
    - OpenAPI
    - Swagger
    - GraphQL
    - Flask Liraries
        - Flask RESTX / RESTPlus (generated specs)
        - Connexion (spec-first)
        - Others?
- Why?
    - ...
- The Problems
    - Inaccurate schemas
        - Undocumented behaviour
    - Invalid schemas
    - Unexpected, but allowed data leads to faults
- Solution
    - Schemathesis
        - Hypothesis, Property Based Testing 
    - Background
        - Swagger Conformance
        - QuickREST
        - …
    - CLI
    - pytest-interface
    - Built in WSGI (and now ASGI) support
        - Import app directly for faster testing
    - HTTP interface
        - Language/framework agnostic universal interface
    - Stateful testing
    - Fixups
    - Targeted property based testing
    - Record into VCR-cassettes
        - Extra fields 
            - command. Full CLI command used to run Schemathesis.
            - http_interactions.id. A numeric interaction ID within the current cassette.
            - http_interactions.status. Type of test outcome, is one of SUCCESS, FAILURE, ERROR.
            - http_interactions.seed. Hypothesis seed used in that particular case could be used as an argument to --hypothesis-seed CLI option to reproduce this request.
            - http_interactions.elapsed. Time in seconds that a request took.
- The Future
- Questions

## API Schemas

API Schemas comes in many flavours, and have become increasingly more popular as our applications are becoming more decoupled. Today we will focus on the types supported by schemathesis, primarly OpenAPI.
- REST/JSON based
    - **OpenAPI 3**
    - Swagger
        - Predecessor to OpenAPI, nowadays a UI to OpenAPI
- GraphQL (Schemathesis support WIP)
    - Typed Query Langague
    - Schema and data format  


### Flask Implementations of Swagger/OpenAPI

There's plenty of libraries implementing OpenAPI-specs for Flask, I won't cover them all in this talk but I'm going to take examples from two approaches.

- **Spec first, generate logic**: [Connexion](https://github.com/zalando/connexion) by Zalando
- **Code first, generate spec**: [Flask-RESTX](https://github.com/python-restx/flask-restx) (replaces [Flask RESTPlus](https://flask-restplus.readthedocs.io/en/stable/)), [Flasgger](https://github.com/flasgger/flasgger), [APISpec](https://github.com/marshmallow-code/apispec), and many more.


## The Problem
Inaccurate data, mismatch between database layer and application layer, library defect, human error, invalid schemas.

**Spectrum of defects**, not all errors are equal but they're never good, here's some examples.
- Incorrect/non conforming schema, lower severity
    - Waste engineering time 😴🥱
    - Leads to incorrect assumptions 🧐
    - Breaks client code generation 🤪
- Unhandled error, lower severity
    - Looks bad 🤨
    - Inconvienience 😔😠
    - Confusion 😕
    - Further escalation? 😈
- Logic errors, medium-high severity  
    - Data corruption 😢
    - Incorrect behaviour 🤔😨
    - Crashed application 🤯🤬
    - Negative/incorrect billing 🧾⚠️📉
- Security problems, high-critical severity
    - Denial of Service (DOS) 🚧⏳💸
    - Data leak 📇💳📤😡
    - Authentication bypass 👺🔓
    - Remote code exectuion (RCE) ⚰️🧨

Many of these problems can be found in tricky corner cases and between layers, and are often missed by traditional testing approaches.

## The Solution
Property based testing (PBT) is great at finding corner casing and lets the computer do much of the heavy lifting in creating exhuastive tests. 

### Hypothesis
The defacto standard for PBT in Python. A property models the behaviour of a piece of code given a certain type of input.

### Modelling properties
To test the application we need to model the properties and their input, but can we do better?  
The nice things with schemas is that they specify the expected behaviour of the application for a defined type of input, sounds a bit like properties, doesn't it?

Can we leverage this? Yes we can with Schemathesis!

## Schemathesis
Takes over where it's spiritual predecessor *Swagger-Conformance* left of, both are based on Hypothesis.
Ispired from the QuickREST research paper.

Automatically generates test cases based on what we already know about or application from our specs.


### What do we already know?
Turns out we already know quite a bit about our application

`r = response`
- r.status_code < 500
- r.status_code in endpoint.responses
- r.headers['Content-Type'] in endpoint.responses[r.status_code]
- r.content matches spec endpoint.responses[r.status_code][r.headers['C…-Type']
- Application should respond
- Server shouldn't crash
- Stateful chains should behave in expected manners (new feature, optional)
    - Create resource
    - Query created resource
    - Update created resource
    - Delete created resource
- etc

## What can schemathesis do more?
Schemathesis can do quite a lot and more is comming!
- CLI
- Built in WSGI (and now ASGI) support
    - Can import app directly for faster testing
    - Uses flask internally for testing
- HTTP interface
    - Language/framework agnostic universal interface
- pytest-interface
- Stateful testing
- Fixups
    - Bultin fixups for FastAPI non-conformance to current OpenAPI spec.
- Hooks (Global, Test, Schema)
    - Customize the behaviour of schemathesis through hooks.
- Targeted property based testing
    - Search for desired goals, combines property based testing with search parameters, reducing randomness.
- Record into VCR-cassettes
    - Extra fields for http_interactions, id, status (SUCCES, FAILURE, ERROR), seed (for `--hypothesis-seed`) and elapsed for timing.
    - Replay recoreded cassettes.

## CLI Interface
Schemathesis implements a rather comprehensive CLI-interface exposing a lot of the functionallity.

`schemathesis run http://127.0.0.1:5000/swagger.json`

### WSGI/ASGI interface vs HTTP interface
I'm a fan of using both, it great to test the entire chain with API-gateways, caching and whatever you may be running in production for a true end to end test.

The WSGI/ASGI interface is useful for quicker local branch testing and for larger amounts of examples/data.

`schemathesis run --app=flask_example:app /swagger.json`

The HTTP Interface is framework (even language) agnostic and can really test and service with a schema specification, be it Python or Cobol.

## pytest
Schemathesis can also be used to generate strategies for pytest where we can define our own properties. 

Previously mentioned built in checks:
- Not a server error (<500)
- Status code conformance
- Content type conformance
- Response schema conformance

Some suggestions to extend schemathesis with
- Complex business rules
- Response time/SLA
- Authentication / 401


From the documentation, tests that any data fitting the schema doesn't cause a server error
```

# test_api.py
import requests
import schemathesis

schema = schemathesis.from_uri("http://0.0.0.0:8080/swagger.json")

@schema.parametrize()
def test_no_server_errors(case):
    # `requests` will make an appropriate call under the hood
    response = case.call()  # use `call_wsgi` if you used `schemathesis.from_wsgi`
    # You could use built-in checks
    case.validate_response(response)
    # Or assert the response manually
    assert response.status_code < 500
```

## Stateful Testing
Shown enhance detection of certain defects in the QuickREST research, and have recently been added to schemathesis.

Schemathesis reuse data from previous requests and responses.  
Resulting tests tend to reach further into the codebase.

Requries `links` between objects, feature came with OpenAPI 3.0, but can be used with 2.0/Swagger with the x-links extension. _(will not work with Flask-RESTX out of the box)_

```
schemathesis run --stateful=links http://0.0.0.0/swagger.yaml

...

POST /api/users/ .                                     [ 33%]
    -> GET /api/users/{user_id} .                      [ 50%]
        -> PATCH /api/users/{user_id} .                [ 60%]
    -> PATCH /api/users/{user_id} .                    [ 66%]
GET /api/users/{user_id} .                             [ 83%]
    -> PATCH /api/users/{user_id} .                    [ 85%]
PATCH /api/users/{user_id} .                           [100%]

...
```

## The Future

- GraphQL
- Schema standard agnostic
- OpenAPI 3.1
- Faster test generation
- Grow the community
- Improve documentation
- …

## Questions

In [1]:
import schemathesis

In [ ]:
schemathesis